## Get first 1000 bases of each gene

In [17]:
import os
import numpy as np
import random
import math

folder = "genes"
print(os.path.join(os.getcwd(), folder))
genes_path = os.path.join(os.getcwd(), folder)
dir_list = os.listdir(genes_path)

data = np.array([])

for file_name in dir_list:
    f = open(os.path.join(genes_path, file_name), "r")
    str1 = ""
    for i, text in enumerate(f):
        if i > 0:
            str1 += text[:len(text) - 1]
    data = np.append(data, str1[:1000])
    f.close()

d:\College\Biomedical Data Mining\Project\genes


Randomly initialize position of motif in each gene

In [18]:
w = 15 # Length of motif
num_genes = len(data)
PWM = np.zeros((w, 4)) # A = 0, C = 1, G = 2, T = 3 (motif length x 4)

bg_prob = np.zeros(4)
count_A = count_C = count_G = count_T = 0
for gene in data:
    for base in gene:
        if base == 'A':
            count_A += 1
        elif base == 'C':
            count_C += 1
        elif base == 'G':
            count_G += 1
        elif base == 'T':
            count_T += 1
bg_prob[0] = count_A / (num_genes * 1000)
bg_prob[1] = count_C / (num_genes * 1000)
bg_prob[2] = count_G / (num_genes * 1000)
bg_prob[3] = count_T / (num_genes * 1000)

start_pos = [random.randint(0, 999 - w + 1) for i in range(num_genes)] # Starting position of the motif in each gene (num_genes)

cur_motif = np.array([]) # Current motif based on start_pos (num_genes x motif length)
for i in range(num_genes):
    cur_motif = np.append(cur_motif, data[i][start_pos[i]:start_pos[i] + w])

Main Loop

In [19]:
it = 0
times_didnt_change = 0

while(True):

    cur_gene = it % num_genes

    # Construct PWM without current gene
    for i in range(w):
        count_A = count_C = count_G = count_T = 0
        for j in range(num_genes):
            if j == cur_gene:
                continue
            if cur_motif[j][i] == 'A':
                count_A += 1
            elif cur_motif[j][i] == 'C':
                count_C += 1
            elif cur_motif[j][i] == 'G':
                count_G += 1
            elif cur_motif[j][i] == 'T':
                count_T += 1
        PWM[i][0] = (count_A + 1) / (num_genes - 1 + 4)
        PWM[i][1] = (count_C + 1) / (num_genes - 1 + 4)
        PWM[i][2] = (count_G + 1) / (num_genes - 1 + 4)
        PWM[i][3] = (count_T + 1) / (num_genes - 1 + 4)

    max_score = 0
    best_pos = 0

    # Check score for each possible motif position, get max score
    for i in range(1000 - w + 1):
        logscore = 0
        for j in range(w):
            if data[cur_gene][i + j] == 'A':
                logscore += math.log(PWM[j][0]/bg_prob[0])
            elif data[cur_gene][i + j] == 'C':
                logscore += math.log(PWM[j][1]/bg_prob[1])
            elif data[cur_gene][i + j] == 'G':
                logscore += math.log(PWM[j][2]/bg_prob[2])
            elif data[cur_gene][i + j] == 'T':
                logscore += math.log(PWM[j][3]/bg_prob[3])
        if logscore > max_score or i == 0:
            max_score = logscore
            best_pos = i

    
    if start_pos[cur_gene] == best_pos: # If the motif position didnt change num_genes times in a row, end loop
        times_didnt_change += 1
    else:
        times_didnt_change = 0

    if times_didnt_change >= num_genes:
        break

    # Update start position and motif
    start_pos[cur_gene] = best_pos
    cur_motif[cur_gene] = data[cur_gene][best_pos:best_pos + w]
    it += 1

print("Iterations: " + str(it))

# Calculate PWM
for i in range(w):
    count_A = count_C = count_G = count_T = 0
    for j in range(num_genes):
        if cur_motif[j][i] == 'A':
            count_A += 1
        elif cur_motif[j][i] == 'C':
            count_C += 1
        elif cur_motif[j][i] == 'G':
            count_G += 1
        elif cur_motif[j][i] == 'T':
            count_T += 1
    PWM[i][0] = (count_A + 1) / (num_genes + 4)
    PWM[i][1] = (count_C + 1) / (num_genes + 4)
    PWM[i][2] = (count_G + 1) / (num_genes + 4)
    PWM[i][3] = (count_T + 1) / (num_genes + 4)

print(PWM)
output_motif = ''
for i in range(w):
    index = np.argmax(PWM[i])
    if index == 0:
        output_motif += 'A'
    elif index == 1:
        output_motif += 'C'
    elif index == 2:
        output_motif += 'G'
    elif index == 3:
        output_motif += 'T'
print(output_motif)

Iterations: 131
[[0.03448276 0.03448276 0.27586207 0.65517241]
 [0.20689655 0.4137931  0.13793103 0.24137931]
 [0.03448276 0.62068966 0.03448276 0.31034483]
 [0.03448276 0.75862069 0.03448276 0.17241379]
 [0.20689655 0.51724138 0.10344828 0.17241379]
 [0.06896552 0.10344828 0.06896552 0.75862069]
 [0.31034483 0.03448276 0.44827586 0.20689655]
 [0.06896552 0.86206897 0.03448276 0.03448276]
 [0.10344828 0.4137931  0.44827586 0.03448276]
 [0.24137931 0.65517241 0.06896552 0.03448276]
 [0.34482759 0.13793103 0.17241379 0.34482759]
 [0.06896552 0.17241379 0.10344828 0.65517241]
 [0.37931034 0.17241379 0.13793103 0.31034483]
 [0.03448276 0.17241379 0.06896552 0.72413793]
 [0.27586207 0.10344828 0.03448276 0.5862069 ]]
TCCCCTGCGCATATT
